### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd
import os

school_file=os.path.join("Resources","schools_complete.csv")
student_file=os.path.join("Resources","students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_file)
student_df = pd.read_csv(student_file)

# Combine the data into a single dataset.  
district_df=student_df.merge(school_df,how="left",on="school_name")
district_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [16]:
#Total Number of Schools
total_schools=district_df["school_name"].unique()
total_schools=len(total_schools)

#Total Number of Students
total_students=len(district_df["student_name"])

#Total Budget
total_budget=school_df["budget"].sum()

#Average Match Score
avg_math=district_df["math_score"].mean()

# Average Reading Score
avg_reading=district_df["reading_score"].mean()

#Percentage of Students with Passing Match Scores (70 or greater)
filter_math=district_df.loc[:,"math_score"]>=70
math_pass=(len(district_df.loc[filter_math,:])/total_students)*100

#Percentage of Students with Passing Reading Score (70 or greater)
filter_reading=district_df.loc[:,"reading_score"]>=70
reading_pass=(len(district_df.loc[filter_reading,:])/total_students)*100

#Percent of Students who Passed Math and Reading
filter_math_reading=(district_df.loc[:,"math_score"]>=70)&(district_df.loc[:,"reading_score"]>=70)
overall_pass=(len(district_df.loc[filter_math_reading,:])/total_students)*100

#District Summary Data Frame

district_summary={"Total Schools":total_schools,
                                "Total Students": [total_students],
                                "Total Budget":[total_budget],
                                 "Average Math Score": [avg_math],
                                 "Average Reading Score": [avg_reading],
                                 "% Students Passing Math": [math_pass],
                                 "% Students Passing Reading": [reading_pass],
                                 "% Overall Students Passing": [overall_pass]
                                }

district_summary_df=pd.DataFrame(district_summary)

#Formatted ditrict_df .map("${:.2f}".format)
district_formatted_df=district_summary_df

district_formatted_df["Total Schools"]=district_formatted_df["Total Schools"]
district_formatted_df["Total Students"]=district_formatted_df["Total Students"].map("{:,.0f}".format)
district_formatted_df["Total Budget"]=district_formatted_df["Total Budget"].map("${:,.2f}".format)
district_formatted_df["Average Math Score"]=district_formatted_df["Average Math Score"].map("{:,.2f}%".format)
district_formatted_df["Average Reading Score"]=district_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
district_formatted_df["% Students Passing Math"]=district_formatted_df["% Students Passing Math"].map("{:,.2f}%".format)
district_formatted_df["% Students Passing Reading"]=district_formatted_df["% Students Passing Reading"].map("{:,.2f}%".format)
district_formatted_df["% Overall Students Passing"]=district_formatted_df["% Overall Students Passing"].map("{:,.2f}%".format)

district_formatted_df.head()


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Students Passing
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary
* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [17]:
# School Name, Type, Size and Budget
school_2_df=school_df[["school_name","type","size","budget"]].set_index('school_name',drop=True)

# Per Student Budget
school_2_df["per_student_budget"]=school_2_df['budget']/school_2_df['size']

#District Data Frame Groupedby School Name Used in by School Calculations
school_group_df=district_df.groupby(['school_name'])

# Average Math Score
school_avg_m=school_group_df['math_score'].mean()

# Average Reading Score
school_avg_r=school_group_df['reading_score'].mean()

#Percentage of Students with Passing Match Scores (70 or greater)
filter_math=district_df.loc[:,"math_score"]>=70
math_pass_group=district_df.loc[filter_math,["math_score","school_name"]].groupby('school_name')                            
math_pass_percent=((math_pass_group["math_score"].count())/(school_group_df['size'].mean()))*100

#Percentage of Students with Passing Reading Score (70 or greater)
filter_reading=district_df.loc[:,"reading_score"]>=70
reading_pass_group=district_df.loc[filter_reading,["reading_score",'school_name']].groupby('school_name')
reading_pass_percent=100*reading_pass_group["reading_score"].count()/(school_group_df['size'].mean())

#Percent of Students who Passed Math and Reading
filter_math_reading_2=(district_df.loc[:,"math_score"]>=70)&(district_df.loc[:,"reading_score"]>=70)
reading_math_pass_group=district_df.loc[filter_math_reading_2,["school_name","math_score"]].groupby('school_name')
reading_math_pass_pct=100*reading_math_pass_group["math_score"].count()/(school_group_df['size'].mean())

#Combined Interim School Summary Combines all Series by School Name into Data Frame
school_summary_dict={"Average Math Score":school_avg_m,
                                "Average Reading Score": school_avg_r,
                                "% Passing Math":math_pass_percent,
                                 "% Passing Reading": reading_pass_percent,
                                 "% Passing Overall": reading_math_pass_pct,
                                }
# school_summary_dict
school_interim_summary_df=pd.DataFrame(school_summary_dict)
# school_interim_summary_df.head()

# Merge School 2 Data Frame with Interim Data Frame

school_interim_summary_2_df=school_2_df.merge(school_interim_summary_df,how='inner',left_index=True,right_index=True)
school_interim_summary_2_df

#School Summary Data Frame with Clear Column Names - cannot use .map to format, data must remain float
school_summary_columns_df=school_interim_summary_2_df.rename(columns={'type':'School Type',
                                                        'size':'Total Students',             
                                                        'budget':'Total Budget',
                                                        'per_student_budget':'Per Student Budget'
                                                             }
                                                    )

school_summary_df=school_summary_columns_df.sort_values(by='school_name')
school_summary_df.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455


## Top Performing Schools (By % Overall Passing)
* Sort and display the top five performing schools by % overall passing.

In [18]:
#Top five performing schools by % overall passing
high_overall_df=school_summary_df.sort_values(by='% Passing Overall', ascending=False)

#Formatted School Summary for Visual, not to be used in any calculations as data has been converted to string
high_overall_formatted_df=high_overall_df.loc[:,:]

high_overall_formatted_df["Total Students"]=high_overall_formatted_df["Total Students"].map("{:,.0f}".format)
high_overall_formatted_df["Total Budget"]=high_overall_formatted_df["Total Budget"].map("${:,.2f}".format)
high_overall_formatted_df["Average Math Score"]=high_overall_formatted_df["Average Math Score"].map("{:,.2f}%".format)
high_overall_formatted_df["Average Reading Score"]=high_overall_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
high_overall_formatted_df["% Passing Math"]=high_overall_formatted_df["% Passing Math"].map("{:,.2f}%".format)
high_overall_formatted_df["% Passing Reading"]=high_overall_formatted_df["% Passing Reading"].map("{:,.2f}%".format)
high_overall_formatted_df["% Passing Overall"]=high_overall_formatted_df["% Passing Overall"].map("{:,.2f}%".format)

high_overall_formatted_df.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",582.0,83.06%,83.98%,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",638.0,83.42%,83.85%,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",625.0,83.35%,83.82%,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",578.0,83.27%,83.99%,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",609.0,83.84%,84.04%,94.59%,95.95%,90.54%


In [19]:
#Observation 1: The top 5 performing schools based on overall passing rate are all charter schools with less than 2300 students, passing rates in math, reading and overall passing rates great than 90%

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [20]:
#Top five lowest schools by % overall passing
low_overall_df=school_summary_df.sort_values(by='% Passing Overall', ascending=True)

#Formatted School Summary for Visual, not to be used in any calculations as data has been converted to string
low_overall_formatted_df=low_overall_df.loc[:,:]

low_overall_formatted_df["Total Students"]=low_overall_formatted_df["Total Students"].map("{:,.0f}".format)
low_overall_formatted_df["Total Budget"]=low_overall_formatted_df["Total Budget"].map("${:,.2f}".format)
low_overall_formatted_df["Average Math Score"]=low_overall_formatted_df["Average Math Score"].map("{:,.2f}%".format)
low_overall_formatted_df["Average Reading Score"]=low_overall_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
low_overall_formatted_df["% Passing Math"]=low_overall_formatted_df["% Passing Math"].map("{:,.2f}%".format)
low_overall_formatted_df["% Passing Reading"]=low_overall_formatted_df["% Passing Reading"].map("{:,.2f}%".format)
low_overall_formatted_df["% Passing Overall"]=low_overall_formatted_df["% Passing Overall"].map("{:,.2f}%".format)

low_overall_formatted_df.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",637.0,76.84%,80.74%,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",639.0,76.71%,81.16%,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",655.0,76.63%,81.18%,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",652.0,77.29%,80.93%,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",650.0,77.07%,80.97%,66.06%,81.22%,53.54%


In [21]:
#Observation 2: The lowest 5 performaing schools based on overall passing rate have more than 2,900 total students, have a greater passing rate in math as compared to reading, and have overall passing rates near 50%, 40% lower than the top 5 performing schools. 

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [22]:
#Create a DataFram that brings in 'grade', 'school_name', 'reading_score', 'math_score'
grade_df=district_df[['grade', 'school_name', 'reading_score', 'math_score']].set_index('school_name')
# grade_df['grade'].unique()
#'9th', '12th', '11th', '10th'

In [23]:
#Average Math Grade by school and grade
#9th Grade
grade_9_filter=grade_df.loc[:,'grade']=='9th'
grade_9_ser=grade_df.loc[grade_9_filter,'math_score']
school_9=grade_9_ser.groupby('school_name').mean()

#10th Grade
grade_10_filter=grade_df.loc[:,'grade']=='10th'
grade_10_ser=grade_df.loc[grade_10_filter,'math_score']
school_10=grade_10_ser.groupby('school_name').mean()

# 11th Grade
grade_11_filter=grade_df.loc[:,'grade']=='11th'
grade_11_ser=grade_df.loc[grade_11_filter,'math_score']
school_11=grade_11_ser.groupby('school_name').mean()

#12th Grade
grade_12_filter=grade_df.loc[:,'grade']=='12th'
grade_12_ser=grade_df.loc[grade_12_filter,'math_score']
school_12=grade_12_ser.groupby('school_name').mean()

#Combine Grade Series Grouped By School Name into Data Frame
avg_math_grade_df=pd.DataFrame({'9th Grade':school_9,'10th Grade': school_10, '11th Grade': school_11,'12th Grade':school_12})

#Formatted Average Math Grade Data Fram for Visual Purposes Only
avg_math_grade_formatted_df=avg_math_grade_df

avg_math_grade_formatted_df["9th Grade"]=avg_math_grade_formatted_df["9th Grade"].map("{:,.2f}%".format)
avg_math_grade_formatted_df["10th Grade"]=avg_math_grade_formatted_df["10th Grade"].map("{:,.2f}%".format)
avg_math_grade_formatted_df["11th Grade"]=avg_math_grade_formatted_df["11th Grade"].map("{:,.2f}%".format)
avg_math_grade_formatted_df["12th Grade"]=avg_math_grade_formatted_df["12th Grade"].map("{:,.2f}%".format)

avg_math_grade_formatted_df.head()

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%


In [24]:
#Observation 3: Average math grades appear to be be consistent across the 9-12 grade levels at each of the respective schools, meaning that math performace is influenced by the school which the student attends and is constant across grades.  

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [25]:
#Average Reding Grade by school and grade
#9th Grade
grade_9_filter=grade_df.loc[:,'grade']=='9th'
r_grade_9_ser=grade_df.loc[grade_9_filter,'reading_score']
r_school_9=r_grade_9_ser.groupby('school_name').mean()

#10th Grade
grade_10_filter=grade_df.loc[:,'grade']=='10th'
r_grade_10_ser=grade_df.loc[grade_10_filter,'reading_score']
r_school_10=r_grade_10_ser.groupby('school_name').mean()

#11th Grade
grade_11_filter=grade_df.loc[:,'grade']=='11th'
r_grade_11_ser=grade_df.loc[grade_11_filter,'reading_score']
r_school_11=r_grade_11_ser.groupby('school_name').mean()

# 12th Grade
grade_12_filter=grade_df.loc[:,'grade']=='12th'
r_grade_12_ser=grade_df.loc[grade_12_filter,'reading_score']
r_school_12=r_grade_12_ser.groupby('school_name').mean()

#Combine Grade Series Grouped By School Name into Data Frame
avg_reading_grade_df=pd.DataFrame({'9th Grade':r_school_9,'10th Grade': r_school_10, '11th Grade': r_school_11,'12th Grade':r_school_12})

#Formatted Average Math Grade Data Fram for Visual Purposes Only
avg_reading_grade_formatted_df=avg_reading_grade_df

avg_reading_grade_formatted_df["9th Grade"]=avg_reading_grade_formatted_df["9th Grade"].map("{:,.2f}%".format)
avg_reading_grade_formatted_df["10th Grade"]=avg_reading_grade_formatted_df["10th Grade"].map("{:,.2f}%".format)
avg_reading_grade_formatted_df["11th Grade"]=avg_reading_grade_formatted_df["11th Grade"].map("{:,.2f}%".format)
avg_reading_grade_formatted_df["12th Grade"]=avg_reading_grade_formatted_df["12th Grade"].map("{:,.2f}%".format)

avg_reading_grade_formatted_df.head()

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%


In [26]:
#Observation 4: Similar to math scores, reading scores are relatively constant across grade levels

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [27]:
#Average Scores and Passing Rates Based on School Summary Data Frame Per Student Budget
# Create the bins in which Data will be held  
bins = [0, 585, 630, 645, 680]

# Create the Names for the Five Bins
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create Bin Column
school_summary_df["Per Student Budget Bin"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_names)

#Groupby Per Student Buget Bin Column and Calculate the Mean for Grades and Passing Rates
budget_bin_df=school_summary_df.loc[:,['Average Math Score','Average Reading Score','% Passing Math',
                                     '% Passing Reading', '% Passing Overall','Per Student Budget Bin']].groupby('Per Student Budget Bin').mean()

#Format Budget Bin Data Frame for Visual Purposes Only
budget_bin_formatted_df=budget_bin_df

budget_bin_formatted_df["Average Math Score"]=budget_bin_formatted_df["Average Math Score"].map("{:,.2f}%".format)
budget_bin_formatted_df["Average Reading Score"]=budget_bin_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
budget_bin_formatted_df["% Passing Math"]=budget_bin_formatted_df["% Passing Math"].map("{:,.2f}%".format)
budget_bin_formatted_df["% Passing Reading"]=budget_bin_formatted_df["% Passing Reading"].map("{:,.2f}%".format)
budget_bin_formatted_df["% Passing Overall"]=budget_bin_formatted_df["% Passing Overall"].map("{:,.2f}%".format)

budget_bin_formatted_df.head()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Per Student Budget Bin,,,,,
<$585,83.46%,83.93%,93.46%,96.61%,90.37%
$585-630,81.90%,83.16%,87.13%,92.72%,81.42%
$630-645,78.52%,81.62%,73.48%,84.39%,62.86%
$645-680,77.00%,81.03%,66.16%,81.13%,53.53%


In [28]:
#Obervation 5: Overall passing rates appear to decrease as the per sudent budget increases, meaning that spending more money per student does not necessarily mean that the student will perform better on standardized testing.

## Scores by School Size

* Perform the same operations as above, based on school size.

In [29]:
#Average Scores and Passing Rates Based on School Summary Data Frame School Sizes

# Create the bins in which Data will be held  
bins_2 = [0, 1000, 2000, 5000]

# Create the names for the five bins
group_names_2 = ["Small (<1000)", "Medium (1000-2000)","Large (2000-5000)"]

#Create School Size Bin Column
school_summary_df["School Size Bin"] = pd.cut(school_summary_df["Total Students"], bins=bins_2, labels=group_names_2)

#Groupby School Size Bin Column and Calculate the Mean for Grades and Passing Rates
size_bin_df=school_summary_df.loc[:,['Average Math Score','Average Reading Score','% Passing Math',
                                     '% Passing Reading', '% Passing Overall','School Size Bin']].groupby('School Size Bin').mean()

#Format School Size Bin Data Frame for Visual Purposes Only
size_bin_formatted_df=size_bin_df

size_bin_formatted_df["Average Math Score"]=size_bin_formatted_df["Average Math Score"].map("{:,.2f}%".format)
size_bin_formatted_df["Average Reading Score"]=size_bin_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
size_bin_formatted_df["% Passing Math"]=size_bin_formatted_df["% Passing Math"].map("{:,.2f}%".format)
size_bin_formatted_df["% Passing Reading"]=size_bin_formatted_df["% Passing Reading"].map("{:,.2f}%".format)
size_bin_formatted_df["% Passing Overall"]=size_bin_formatted_df["% Passing Overall"].map("{:,.2f}%".format)

size_bin_formatted_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size Bin,,,,,
Small (<1000),83.82%,83.93%,93.55%,96.10%,89.88%
Medium (1000-2000),83.37%,83.86%,93.60%,96.79%,90.62%
Large (2000-5000),77.75%,81.34%,69.96%,82.77%,58.29%


In [30]:
#Observation 6: Schools with more than 2000 students see a drastic (nealry 32%) decrease in overall passing rates as compared to medium and smaller schools

## Scores by School Type

* Perform the same operations as above, based on school type

In [31]:
#Groupby School Type and calculate Means for Grades and Passing Rates
type_df=school_summary_df.loc[:,['Average Math Score','Average Reading Score','% Passing Math',
                                     '% Passing Reading', '% Passing Overall','School Type']].groupby('School Type').mean()

#Format School School Type Data Frame for Visual Purposes Only
type_formatted_df=type_df

type_formatted_df["Average Math Score"]=type_formatted_df["Average Math Score"].map("{:,.2f}%".format)
type_formatted_df["Average Reading Score"]=type_formatted_df["Average Reading Score"].map("{:,.2f}%".format)
type_formatted_df["% Passing Math"]=type_formatted_df["% Passing Math"].map("{:,.2f}%".format)
type_formatted_df["% Passing Reading"]=type_formatted_df["% Passing Reading"].map("{:,.2f}%".format)
type_formatted_df["% Passing Overall"]=type_formatted_df["% Passing Overall"].map("{:,.2f}%".format)

type_formatted_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,90.43%
District,76.96%,80.97%,66.55%,80.80%,53.67%


In [32]:
#Observation 7: Charter schools have a higher math and reading grade averages as well as higher passing rates across the board compared to district schools.